In [1]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException

In [3]:
# Path to your latest browser driver.
driver_path = 'C:\Program Files (x86)\chromedriver.exe'

# Link of the university you want to scrape data
links = ['https://yocket.in/university-reviews/arizona-state-university-782/']

# Streams You are intereseted in the given university
streams = ['engineering','business-and-management','medicine','sciences','arts-and-humanities','architecture','law']


In [3]:
data = pd.read_csv('Yocket_Universities_link.csv')
data.head(10)

,University,Link,Courses offered,Fees
0,Massachusetts Institute of Technology (MIT),https://yocket.in/university-reviews/massachus...,67 Courses,$49580 Tuition*
1,Technical University of Munich (TU München),https://yocket.in/university-reviews/technical...,102 Courses,€12940 Tuition*
2,University of Toronto (U Toronto),https://yocket.in/university-reviews/universit...,143 Courses,C$44000 Tuition*
3,University of Melbourne (Uni Melb),https://yocket.in/university-reviews/universit...,215 Courses,A$39520 Tuition*
4,National University Of Singapore (NUS),https://yocket.in/university-reviews/national-...,72 Courses,S$38200 Tuition*
5,Nanyang Technological University (NTU),https://yocket.in/university-reviews/national-...,41 Courses,S$16700 Tuition*
6,University of Auckland (U Auckland),https://yocket.in/university-reviews/universit...,47 Courses,NZ$35000 Tuition*
7,The University of Tokyo (Utokyo),https://yocket.in/university-reviews/the-unive...,25 Courses,¥1100000 Tuition*
8,University of Glasgow (GLA),https://yocket.in/university-reviews/universit...,81 Courses,£19500 Tuition*
9,University of Cambridge (Cambridge),https://yocket.in/university-reviews/universit...,73 Courses,£21600 Tuition*


In [17]:
links = data.iloc[:2,1]
for i in range(len(links)):
    links[i] = re.sub('/engineering$','/',links[i])
links[1]

'https://yocket.in/university-reviews/technical-university-of-munich-2908/'

In [2]:
class login():
    def __init__(self,URL,driver):
        self.url =URL
        self.driver = driver
        
    def visit(self):
        self.driver.get(self.url)
        
    def email(self,your_mail):
        text_field= self.driver.find_element_by_xpath('//input[@name ="email"]')
        text_field.send_keys(your_mail)  
    
    def submit(self):
        self.driver.find_element_by_xpath('//button[@type="submit"]').click()
        return(driver)
        
    def Password(self,passkey):
        text_field = self.driver.find_element_by_xpath('//input[@name ="password"]')
        text_field.send_keys(passkey)

In [20]:
driver = webdriver.Chrome(driver_path)
wait = WebDriverWait(driver,10)
page = login('https://yocket.in/account/login',driver)
page.visit()
time.sleep(5)

page.email('kalpeshmulye@gmail.com')
page.submit()
time.sleep(2)

page.Password('kalpu22199798')
page.submit()
time.sleep(5)

for link in links:        
    # Loading University URL
    driver.get(link)
    time.sleep(10)
    
    # Getting University name
    University = driver.find_element_by_xpath('//h1')
    University_name = University.text
    
    #Opening a csv file to write 
    file_name = University_name + '.csv'
    f = open(file_name,'w',encoding='utf-8')
    
    # Getting the Generic info of the University
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"[class='col-sm-3 col-xs-6']")))
    Generic = driver.find_elements_by_xpath('//div[@class="row text-center"]/div[@class="col-sm-3 col-xs-6"]/h3')
    for info in Generic[:-1]:
        tmp = info.text.split('\n')
        if info != Generic[-2]:
            f.write(tmp[0].replace(',','') + ' ' + tmp[1] + ',')
        else:
            f.write(tmp[0].replace(',','') + ' ' + tmp[1] + '\n')
    f.write('\n')
    
    for stream in streams:
        try:
            driver.get(link + stream)
            time.sleep(10)
            f.write(stream.upper() + '\n')
            f.write('\n')
            # Getting Header Ready
            Head = driver.find_elements_by_xpath('//table[@class="table table-bordered university-course-list"]/thead/tr/th')
            Header = []
            Header = [ii.text for ii in Head]

            for head in Header[:-1]:
                f.write(head + ',')

            f.write(Header[-1] + '\n')

            #Getting course name, links and Deadlines
            Rows = driver.find_elements_by_xpath('//table[@class="table table-bordered university-course-list"]/tbody/tr')

            for i in range(len(Rows)):
                course = driver.find_elements_by_xpath('//table[@class="table table-bordered university-course-list"]/tbody/tr['+str(i+1)+']/td[1]/li//a')
                description = driver.find_element_by_xpath('//table[@class="table table-bordered university-course-list"]/tbody/tr['+str(i+1)+']/td[2]').text
                Course_name = course[0].text
                Course_link = course[0].get_attribute('href')
                deadline = '//table[@class="table table-bordered university-course-list"]/tbody/tr['+str(i+1)+']/td[1]/li/small/span'
                try:
                    Deadline = driver.find_element_by_xpath(deadline).text
                except :   
                    Deadline = 'Not Available'
                    pass


                f.write(Course_name.replace(',',';') + ',')
                f.write(description.replace(',',';') + ',')
                f.write(Deadline.replace(',',';') + ',')
                f.write(Course_link + '\n')
                f.write('\n')

            time.sleep(5)
        except:
            continue

driver.quit()